### Convert SDO/AIA data from level 1 to level 1.5

AIA data products provides by the JSOC are level 1 data products.  
This means that the images still include the roll angle of the satellite and each channel may have a slightly different pixel scale.  
Typically, before performing any sort of data analysis on AIA images, you will want to promote your AIA data from level 1 to level 1.5.

1. Pointing correction (aiapy.calibrate.update_pointing)  
2. Image respiking (aiapy.calibrate.respike)  
3. PSF deconvolution (aiapy.psf.deconvolve)  
4. Registration (aiapy.calibrate.register)  
5. Degradation correction (aiapy.calibrate.correct_degradation)  
6. Exposure normalization

In this code, we only use method 1, 4, 5, and 6.

Reference.
https://aiapy.readthedocs.io/en/stable/preparing_data.html

In [1]:
import copy
import warnings

import numpy as np

import astropy.units as u

import sunpy
from sunpy.map import contains_full_disk

import aiapy
from aiapy.calibrate.util import get_pointing_table
from aiapy.calibrate.util import get_correction_table
from aiapy.util.exceptions import AIApyUserWarning

__all__ = ["update_pointing"]

In [2]:
fits_file = r"E:\Research\SR\input\CH_Indices\EUV_level1\193\2024\aia.lev1_euv_12s.2024-11-28T120006Z.193.image_lev1.fits"
aia_map = sunpy.map.Map(fits_file)
type(aia_map)

sunpy.map.sources.sdo.AIAMap

#### 1. Pointing correction

aiapy.calibrate.update_pointing(smap, *, pointing_table)

pointing_table = ``get_pointing_table``  
: This method removes any ``PCi_j`` matrix keys in the header and updates the ``CROTA2`` keyword.

In [3]:
pointing_tbl = get_pointing_table(
    "lmsal",
    time_range=(aia_map.date - 6*u.hour, aia_map.date + 6*u.hour)
)

aia_map_pt = aiapy.calibrate.update_pointing(aia_map, pointing_table=pointing_tbl)

In [4]:
aia_map_pt = aiapy.calibrate.update_pointing(aia_map, pointing_table=pointing_tbl)

In [5]:
# To check a change.
for key in ["CRPIX1","CRPIX2","CDELT1","CDELT2","CROTA2"]:
    print(key, "before:", aia_map.meta[key], "after:", aia_map_pt.meta[key])


CRPIX1 before: 2041.36206 after: 2041.550781
CRPIX2 before: 2040.86011 after: 2040.865112
CDELT1 before: 0.600714028 after: 0.600714
CDELT2 before: 0.600714028 after: 0.600714
CROTA2 before: 0.0575987138 after: 0.057598713147


#### 2. Registration

In [6]:
aia_map_reg = aiapy.calibrate.register(
    aia_map_pt,
    missing=np.nan,     # extrapolation: fill with NaN
    order=3,            # interpolation: bicubic        
    method='scipy'      # Rotation function to use: scipy
)

In [7]:
# To check a change.
for key in ["CRPIX1","CRPIX2","CDELT1","CDELT2"]:
    print(key, "before:", aia_map_pt.meta[key], "after:", aia_map_reg.meta[key])

CRPIX1 before: 2041.550781 after: 2048.5
CRPIX2 before: 2040.865112 after: 2048.5
CDELT1 before: 0.600714 after: 0.6
CDELT2 before: 0.600714 after: 0.6


#### 3. Degradation correction

In [8]:
corr_tbl = get_correction_table("SSW")

aia_map_cal = aiapy.calibrate.correct_degradation(
    aia_map_reg,
    correction_table=corr_tbl
)

In [9]:
print("before:", aia_map_reg.date.isot, " after:", aia_map_cal.date.isot)

before: 2024-11-28T12:00:04.835  after: 2024-11-28T12:00:04.835


#### 4. Exposure normalization

In [10]:
exp_time = aia_map_cal.exposure_time

aia_map_norm = aia_map_cal / exp_time

---

##### LMSAL 소스를 이용하여 Master Pointing Table 저장.

In [1]:
from sunpy.time import parse_time
from pathlib import Path
from tqdm import tqdm
from datetime import datetime
import sunpy

from convert_to_level1_5 import convert_to_level1_5


import time


In [8]:
# parameters
channels = [193, 211]
start_dt = parse_time("2012-01-01").to_datetime()
end_dt = parse_time("2013-12-31").to_datetime()
file_directory = r"E:\Research\SR\input\CH_Indices\EUV_level1"
save_directory = r"D:\Research_data\EUV"
parent_dir = Path(file_directory)
save_dir   = Path(save_directory)

years = range(start_dt.year, end_dt.year + 1)

def strip_invalid_blank(aia_map):
    """
    Remove the BLANK keyword when BITPIX < 0 (float data),
    to avoid astropy VerifyWarning.
    """
    if aia_map.meta.get("BITPIX", 0) < 0 and "BLANK" in aia_map.meta:
        aia_map.meta.pop("BLANK") 

for chan in channels:
    for year in years:
        source_dir = parent_dir / str(chan) / str(year)
        destination_dir = save_dir / str(chan)/ str(year)
        destination_dir.mkdir(parents=True, exist_ok=True)

        source_files = sorted(source_dir.glob("*.fits"))
        destination_files = []

        with tqdm(source_files, desc=f"EUV {chan} | year={year}", unit="file") as pbar:
            for file in pbar:
                pbar.set_postfix(file=file.name[:60])
                try:
                    file_date = datetime.strptime(file.stem.split(".")[2],
                                               "%Y-%m-%dT%H%M%SZ")
                except Exception:
                    continue
                if not (start_dt <= file_date <= end_dt):
                    continue
                outfile = destination_dir / file.name.replace("lev1", "lev1_5")
                if outfile.exists():
                    continue

                destination_files.append((str(file), str(outfile)))
                try:
                    t0 = time.perf_counter()
                    aia_map = sunpy.map.Map(file)
                    t1 = time.perf_counter()
                    aia_map_new = convert_to_level1_5(aia_map)
                    t2 = time.perf_counter()
                    strip_invalid_blank(aia_map_new)
                    t3 = time.perf_counter()
                    aia_map_new.save(outfile, overwrite=False)
                    t4 = time.perf_counter()
                    print(f"{file.name}: "
                        f"Map: {t1-t0:.2f}s, "
                        f"Convert: {t2-t1:.2f}s, "
                        f"strip_invalid_blank: {t3-t2:.2f}s, "
                        f"Save: {t4-t3:.2f}s")
                except Exception as e:
                    pbar.write(f"[{chan}] {file.name} failed: {e}")
print("All conversions finished.")

EUV 193 | year=2012:   0%|          | 1/733 [00:06<1:24:21,  6.91s/file, file=aia.lev1_euv_12s.2012-01-01T120009Z.193.image_lev1.fits]

aia.lev1_euv_12s.2012-01-01T000009Z.193.image_lev1.fits: Map: 0.27s, Convert: 5.91s, strip_invalid_blank: 0.00s, Save: 0.73s


EUV 193 | year=2012:   0%|          | 2/733 [00:13<1:24:25,  6.93s/file, file=aia.lev1_euv_12s.2012-01-02T000009Z.193.image_lev1.fits]

aia.lev1_euv_12s.2012-01-01T120009Z.193.image_lev1.fits: Map: 0.27s, Convert: 5.94s, strip_invalid_blank: 0.00s, Save: 0.73s


EUV 193 | year=2012:   0%|          | 3/733 [00:20<1:24:35,  6.95s/file, file=aia.lev1_euv_12s.2012-01-02T120009Z.193.image_lev1.fits]

aia.lev1_euv_12s.2012-01-02T000009Z.193.image_lev1.fits: Map: 0.27s, Convert: 5.98s, strip_invalid_blank: 0.00s, Save: 0.73s


EUV 193 | year=2012:   0%|          | 3/733 [00:22<1:30:37,  7.45s/file, file=aia.lev1_euv_12s.2012-01-02T120009Z.193.image_lev1.fits]


KeyboardInterrupt: 

In [ ]:
"""
Run the conversion of SDO/AIA level 1 FITS files to level 1.5.

Convert SDO/AIA level 1 FITS files to level 1.5 and save them
with a progress bar that shows the number of files processed.

"""

import argparse
from pathlib import Path
from tqdm import tqdm

import sunpy
from sunpy.time import parse_time
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed

from convert_to_level1_5 import convert_to_level1_5


def strip_invalid_blank(aia_map):
    """
    Remove the BLANK keyword when BITPIX < 0 (float data),
    to avoid astropy VerifyWarning.
    """
    if aia_map.meta.get("BITPIX", 0) < 0 and "BLANK" in aia_map.meta:
        aia_map.meta.pop("BLANK") 

def process_and_save(infile: str, outfile: str):
    """
    Save a FITS file after converting to level 1.5
    """
    aia_map = sunpy.map.Map(infile)
    aia_map_new = convert_to_level1_5(aia_map)
    strip_invalid_blank(aia_map_new)
    aia_map_new.save(outfile, overwrite=False)

def main():
    parser = argparse.ArgumentParser(
        description="Convert SDO/AIA data from level 1 to level 1.5."
    )
    parser.add_argument("--channel", type=str, required=True,
                        help="channel name (e.g, '193' or '193,211')")
    parser.add_argument("--start", type=str, required=True,
                        help="start date (e.g, 2012-01-01)")
    parser.add_argument("--end", type=str, required=True,
                        help="end date (e.g, 2024-12-31)")
    parser.add_argument("--file_directory", type=str, required=True,
                        help="directory containing level 1 FITS files (e.g, E:\Research\SR\input\CH_Indices\EUV_level1)")
    parser.add_argument("--save_directory", type=str, required=True,
                        help="directory to save a level 1.5 FITS files (e.g, D:\Research_data\EUV)")
    parser.add_argument("--cores", type=int, default=4,
                        help="number of cores to use for processing")
    args = parser.parse_args()

    parent_dir = Path(args.file_directory)
    save_dir   = Path(args.save_directory)
    
    start_dt = parse_time(args.start).to_datetime()
    end_dt = parse_time(args.end).to_datetime()

    channels = [chan.strip() for chan in args.channel.split(',')]   # e.g., [193,211]
    years = range(start_dt.year, end_dt.year + 1)

    for chan in channels:
        for year in years:
            source_dir = parent_dir / str(chan) / str(year)
            destination_dir = save_dir / str(chan)/ str(year)
            destination_dir.mkdir(parents=True, exist_ok=True)

            destination_files = []
            for file in sorted(source_dir.glob("*.fits")):
                try:
                    file_dt = datetime.strptime(file.stem.split(".")[2],
                                                "%Y-%m-%dT%H%M%SZ")
                except Exception:
                    continue
                if not (start_dt <= file_dt <= end_dt):
                    continue

                outpath = destination_dir / file.name.replace("lev1", "lev1_5")
                if outpath.exists():
                    continue

                destination_files.append((str(file), str(outpath)))

            if not destination_files:
                continue

            with ProcessPoolExecutor(max_workers=args.workers) as executor:
                futures = {
                    executor.submit(process_and_save, inp, outp): (inp, outp)
                    for inp, outp in destination_files
                }

                for future in tqdm(as_completed(futures),
                                   total=len(futures),
                                   desc=f"EUV {chan} | year={year}",
                                   unit="file"):
                    inp, outp = futures[future]
                    try:
                        future.result()
                    except Exception as e:
                        tqdm.write(f"[ERROR] {Path(inp).name} -> {e}")




            source_files = sorted(source_dir.glob("*.fits"))
            destination_files = []

            with tqdm(source_files, desc=f"EUV {chan} | year={year}", unit="file") as pbar:
                for file in pbar:
                    pbar.set_postfix(file=file.name[:60])
                    try:
                        file_date = datetime.strptime(file.stem.split(".")[2],
                                               "%Y-%m-%dT%H%M%SZ")
                    except Exception:
                        continue
                    if not (start_dt <= file_date <= end_dt):
                        continue
                    outfile = destination_dir / file.name.replace("lev1", "lev1_5")
                    if outfile.exists():
                        continue

                    destination_files.append((str(file), str(outfile)))
                    try:
                        aia_map = sunpy.map.Map(file)
                        aia_map_new = convert_to_level1_5(aia_map)
                        strip_invalid_blank(aia_map_new)
                        aia_map_new.save(outfile, overwrite=False)
                    except Exception as e:
                        pbar.write(f"[{chan}] {file.name} failed: {e}")
    print("All conversions finished.")

if __name__ == '__main__':
    main()


# To run this script, you can use the command line as follows:
# conda activate venv
# cd Research\SR_SWspeed\data\CH_Indices\calibration
# python run_convert_to_level1_5.py --channel "193,211" --start "2012-01-01" --end "2023-12-31" --file_directory "E:\Research\SR\input\CH_Indices\EUV_level1" --save_directory "D:\Research_data\EUV"

ModuleNotFoundError: No module named 'fitsio'